<a href="https://colab.research.google.com/github/DouglasAmaral0/BinanceBot/blob/main/MinecraftLLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers
!pip install -q -U datasets
!pip install -q -U accelerate
!pip install -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_

In [2]:
import os
from google.colab import drive

# Monta seu Google Drive no ambiente do Colab
drive.mount('/content/drive',  force_remount=True)

# --- ATENÇÃO: MUDE AQUI SE NECESSÁRIO ---
# Verifique se este caminho corresponde à pasta que você criou no seu Google Drive
BASE_FOLDER = "/content/drive/MyDrive/Colab Notebooks/GPTMinecraft/"
# -----------------------------------------

# Define os caminhos para os arquivos de treino e o diretório de saída
TRAIN_FILE = os.path.join(BASE_FOLDER, "train.txt")
VALID_FILE = os.path.join(BASE_FOLDER, "valid.txt")
OUTPUT_DIR = os.path.join(BASE_FOLDER, "mistral-7b-minecraft-agent")

# Verifica se os arquivos existem para evitar erros
assert os.path.exists(TRAIN_FILE), f"Arquivo de treino não encontrado em: {TRAIN_FILE}"
assert os.path.exists(VALID_FILE), f"Arquivo de validação não encontrado em: {VALID_FILE}"

Mounted at /content/drive


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# O nome do modelo que vamos usar do Hugging Face
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Configuração de Quantização (QLoRA)
# Isso permite carregar o modelo gigante em 4 bits, economizando muita memória
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Carrega o modelo com a configuração de quantização
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # Mapeia o modelo para a GPU automaticamente
)

# Carrega o tokenizador do modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Adiciona um token de padding para que todas as sequências tenham o mesmo tamanho
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
special_tokens = ["[TAREFA]", "[INVENTÁRIO]", "[AÇÃO]"]
tokenizer.add_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32003, 4096)

In [4]:
from peft import LoraConfig, get_peft_model

# Configuração do LoRA
# Diz ao programa quais partes do modelo "congelado" devem ser adaptadas
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Aplica a configuração LoRA ao modelo
model = get_peft_model(model, lora_config)

In [7]:
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

# Carrega os datasets de texto a partir dos seus arquivos
dataset = load_dataset('text', data_files={'train': TRAIN_FILE, 'validation': VALID_FILE})

# Define os argumentos de treinamento
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,  # Reduzido para economizar memória
    gradient_accumulation_steps=8,  # Aumentado para compensar
    learning_rate=2e-4,
    num_train_epochs=20,
    logging_steps=5,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,  # Mantém apenas os 3 melhores checkpoints
    warmup_ratio=0.1,    # Aquecimento gradual do learning rate
    lr_scheduler_type="cosine",  # Scheduler mais suave
    report_to="none"
)
training_arguments = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,  # Reduzido - seus exemplos são curtos
    packing=False,       # Desabilitado para melhor controle
)
# Cria o SFTTrainer (Supervised Fine-tuning Trainer)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=lora_config, # O nome da coluna com o texto
    args=training_args,
    processing_class=tokenizer
)

Converting train dataset to ChatML:   0%|          | 0/130 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/130 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/130 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/130 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/57 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/57 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/57 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/57 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
import json
trainer.train()
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
config = {
        "base_model": model_id,
        "task_type": "minecraft_agent",
        "special_tokens": special_tokens,
        "max_seq_length": 512
        }
with open(os.path.join(OUTPUT_DIR, "training_config.json"), "w") as f: json.dump(config, f, indent=2)

print(f"Modelo completo salvo em: {OUTPUT_DIR}")


Epoch,Training Loss,Validation Loss
1,4.407500,1.862705
2,1.459000,1.360412
3,1.114100,1.038946
4,0.931200,0.979049
5,0.871300,0.963932
6,0.895300,1.033455
7,0.868600,0.962267
8,0.876200,0.999334
9,0.844100,0.964061
10,0.818500,0.926847


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

Modelo completo salvo em: /content/drive/MyDrive/Colab Notebooks/GPTMinecraft/mistral-7b-minecraft-agent


In [21]:
from transformers import pipeline

tarefa = "criar espada de ferro"
inventario = '{"pedregulho": 5, "graveto": 1}'

"""Testa o modelo de forma mais robusta"""
prompt = f"[TAREFA] {tarefa} [INVENTÁRIO] {inventario} [AÇÃO]"

    # Tokeniza de forma mais controlada
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_device = trainer.model.device
inputs = {k: v.to(model_device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = trainer.model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1
    )

# Decodifica apenas a parte nova
new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
response = tokenizer.decode(new_tokens, skip_special_tokens=True)

# Limpa a resposta
response = response.split('<|endoftext|>')[0].strip()

print(f"Prompt: {prompt}")
print(f"Resposta: {response}")


Prompt: [TAREFA] criar espada de ferro [INVENTÁRIO] {"pedregulho": 5, "graveto": 1} [AÇÃO]
Resposta: obter barras_de_ferro


In [13]:
import torch
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import os

# --- 1. CONECTAR AO DRIVE E DEFINIR CAMINHOS ---
drive.mount('/content/drive',  force_remount=True)

BASE_FOLDER = "/content/drive/MyDrive/Colab Notebooks/GPTMinecraft/"
MODEL_OUTPUT_DIR = os.path.join(BASE_FOLDER, "mistral-7b-minecraft-agent")

# --- 2. CARREGAR O MODELO BASE E O MODELO TREINADO ---
# Precisamos primeiro carregar o Mistral 7B original em 4-bit
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# --- VERIFICAÇÃO DE DISPOSITIVO ---
if torch.cuda.is_available():
    print("GPU detectada. Carregando o modelo com bitsandbytes (4-bit quantization).")
    # Configuração de Quantização (QLoRA) para GPU
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        device_map="auto" # Mapeia o modelo para a GPU automaticamente
    )
else:
    print("GPU não detectada. Carregando o modelo na CPU em precisão total (pode exigir bastante RAM).")
    # Carrega o modelo base sem quantização para CPU
    # Você pode tentar torch_dtype=torch.bfloat16 ou torch.float16 se quiser,
    # mas a compatibilidade varia na CPU e float32 é o padrão e mais seguro.
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch.bfloat16 # Descomente se quiser tentar bfloat16 na CPU
    )

# Agora, carregamos nosso adaptador LoRA treinado e o fundimos ao modelo base
print("Carregando o modelo treinado (adaptador LoRA)...")
model = PeftModel.from_pretrained(base_model, MODEL_OUTPUT_DIR)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

print("Modelo final carregado com sucesso!")

KeyboardInterrupt: 

In [ ]:


# --- 3. EXECUTAR O TESTE FINAL ---
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=10 # Só precisamos da primeira ação, não precisa ser longo
)

# O teste da lógica quantitativa
tarefa = "criar espada_de_pedra"
inventario = '{"pedregulho": 5, "graveto": 1}'
prompt = f"[TAREFA] {tarefa} [INVENTÁRIO] {inventario} [AÇÃO]"

# Gera a resposta
result = pipe(prompt)

# Exibe a resposta
print("\n--- PROMPT ENVIADO ---")
print(prompt)
print("\n--- RESPOSTA GERADA ---")
print(result[0]['generated_text'])

Mounted at /content/drive
GPU detectada. Carregando o modelo com bitsandbytes (4-bit quantization).


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Carregando o modelo treinado (adaptador LoRA)...


Device set to use cuda:0


Modelo final carregado com sucesso!

--- PROMPT ENVIADO ---
[TAREFA] criar espada_de_pedra [INVENTÁRIO] {"pedregulho": 5, "graveto": 1} [AÇÃO]

--- RESPOSTA GERADA ---
[TAREFA] criar espada_de_pedra [INVENTÁRIO] {"pedregulho": 5, "graveto": 1} [AÇÃO] craft espada_de_pedra
